In [1]:
import numpy as np
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
# This import WILL work after you fix your environment
from qiskit_nature.second_q.transformers import QubitConverter 
from qiskit_nature.second_q.formats import Molecule
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler

# --- 1. Define the H2 Molecule (Problem) ---
print("Setting up H2 molecule...")

# Define the molecule at its equilibrium bond length (0.74 Å)
mol = Molecule(
    atoms=[("H", [0.0, 0.0, 0.0]), ("H", [0.0, 0.0, 0.74])],
    charge=0,
    multiplicity=1,
)

# Use PySCFDriver to run the classical chemistry calculation
driver = PySCFDriver(molecule=mol, basis="sto3g")
problem = driver.run()

# --- 2. Map to a Qubit Hamiltonian ---
print("Mapping to qubit Hamiltonian...")

# Use Jordan-Wigner mapping
mapper = JordanWignerMapper()

# QubitConverter simplifies the Hamiltonian
converter = QubitConverter(mapper, two_qubit_reduction=True, z2symmetries=problem.z2symmetries)

# Convert the fermionic Hamiltonian to a qubit Hamiltonian
qubit_op = converter.convert(
    problem.second_q_ops()["ElectronicEnergy"],
    num_particles=problem.num_particles,
)

print(f"\nNumber of qubits after tapering: {qubit_op.num_qubits}")
print("Tapered Hamiltonian:")
print(qubit_op)

# --- 3. Set up QAOA Algorithm ---
print("\nConfiguring QAOA...")

# COBYLA is a good gradient-free optimizer
optimizer = COBYLA(maxiter=100)

# Sampler runs the quantum circuits (we'll use the local simulator)
sampler = Sampler()

# `reps=p` is the number of layers in the QAOA circuit.
p_layers = 2

# Instantiate the QAOA eigensolver
qaoa = QAOA(sampler=sampler, optimizer=optimizer, reps=p_layers)

# --- 4. Run QAOA and Find Ground State ---
print(f"Running QAOA with p={p_layers} layers...")

# compute_minimum_eigenvalue finds the ground state
result = qaoa.compute_minimum_eigenvalue(qubit_op)

print("\n--- Results ---")

# The 'result' object contains the raw eigenvalue of the qubit_op.
# We use problem.interpret() to add back the tapered-off energy
# and the classical nuclear repulsion energy to get the true total energy.
final_result = problem.interpret(result)

# The exact classical energy (Full Configuration Interaction)
fci_energy = problem.reference_energy

print(f"QAOA Ground State Energy: {final_result.total_energies[0]:.6f} Ha")
print(f"FCI (Exact) Energy:     {fci_energy:.6f} Ha")
print(f"Error:                    {np.abs(final_result.total_energies[0] - fci_energy):.6f} Ha")

ImportError: cannot import name 'QubitConverter' from 'qiskit_nature.second_q.transformers' (/root/miniconda3/envs/qiskit-env/lib/python3.11/site-packages/qiskit_nature/second_q/transformers/__init__.py)

In [1]:
!pip list |grep qiskit

qiskit                    2.0.1
qiskit-ibm-runtime        0.39.0
qiskit-qasm3-import       0.6.0


In [2]:
!pip install --upgrade --force-reinstall qiskit qiskit-algorithms qiskit-nature pyscf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 7.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 KB 8.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.7 MB/s eta 0:00:0000:0100:01
  Using cached pyscf-2.11.0-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (51.3 MB)
  Using cached rustworkx-0.17.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.2 MB)
  Using cached stevedore-5.5.0-py3-none-any.whl (49 kB)
  Using cached scipy-1.15.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.7 MB)
  Using cached dill-0.4.0-py3-none-any.whl (119 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl (44 kB)
  Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.3 MB/s eta 0:00:0000:0100:01
  Using cached h5py-3.15.1-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (4.